# Segmenting and Clustering Neighborhoods in Harare

### Introduction

Harare formerly Salisbury until (1982) is the capital and most populous city of Zimbabwe. The city proper has an area of 960.6 km2 (371 mi2) and a population 2,123,132 in the 2012 census and an estimated 3,120,917 in its metropolitan area in 2019 (Zimstats 2012). Situated in north-eastern Zimbabwe in the country's Mashonaland region, Harare is a metropolitan province, which also incorporates the municipalities of Chitungwiza and Epworth. The city sits on a plateau at an elevation of 1,483 metres (4,865 feet) above sea level and its climate falls into the subtropical highland category.

Long the commercial capital of Zimbabwe, Harare has seen economic ups and downs since the 2000s. It remains an important centre of commerce, government, finance, real estate, manufacturing, healthcare, design, education, art, culture, tourism, agriculture, mining and regional affairs. Individuals engaged in this particular activities or in these suburbs need to know the nearest place they can get certain ammenities. 

### Objective 

It is clear that Harare is the financial centre of the country. It has been well know that the Northern and North Eastern suburbs of Harare are home to the more affluent population of the city. These northern suburbs are often referred to as 'dales' because of the common suffix -dale found in some suburbs such as Avondale, Greendale and Borrowdale. With that in mind, the project aims to:

1. To cluster Harare Surbubs based on places to go for social gethering and/or get ammenities
2. To find the most popular social gathering and/or ammenities in each neighborhood


### Data Description

1. To achive our objective. We need data. Data for Zimbabwe not readily available on the internet...but i got lucky, I found the awesome list with all the surbubs of Harare. http://archive.kubatana.net/docs/elec/020228hrepollcon.pdf. It was mainly created to list down the relevent places that make up the voting population of Harare (..still serves the pupose of me).Notice how its downloading a .pdf file? Yup, i noticed that too so i used excel to extract the date and put it in a fomat pandas understands.


2. I have a list of neighborhoods but you guys from (..not from here) dont know where these places are. Well, Google is here to save the day. To get the latitude and longitudes, i used the Google Geocoding API https://developers.google.com/maps/documentation/geocoding/overview . Afterwards I joined the latitude and longitude to the data frame.


3. To be honest, I didnt even know what Foursquare was before this project. I am not ignorant, in my defense, how do you know what the Fibonacci Sequence is when you cant even add 1 + 1? Ahaha, i digress. I used the Foursquare API to explore neighborhoods in of Harare. I used the explore function to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters.

Before we jump into out analysis, we need to load our libraries that we will be using in this project. 

### Methodology 

Data Collection Method: Data was download from the web in a pdf fomat. It put into excel and was cleaned and saved down in a csv format. 

Geographic Location Data Collection Method: The initial data frame has been used to get longitude and latitude of neighborhoods on the Googlemaps library. 

Map Exploration Method: The Folium library has been utilized to get the map of Harare.

Nearby Location Data Extraction Methodology: Folium is a powerful library provides nearby venues such as Yoga Studio, Afghan Restaurant, Airport, Airport Food Court, Airport Gate, Airport Lounge, Airport Service, Airport Terminal, American Restaurant, Antique Shop, Thrift / Vintage Store, Toy / Game Store, 'Trail, Train Station, Vegetarian / Vegan Restaurant, Video Game Store, Vietnamese Restaurant, Wine Bar, Wings Joint, and Women's Store.

K-means clustering- K means cluster is applied to the dataset for the exploration of different venues. We used the elbow method to determine the optimal K. Here the algorithm formed 4 clusters with the trends. In the first step we have collected the required data: location and type (category) of every restaurant within 6km from Harare center . We have also *identified different * (according to Foursquare categorization). We have also presented the top 10 venue categories for each cluster.

We will present a map of all such locations but also create clusters (using k-means clustering) of those locations to identify general zones/neighborhoods/addresses which should be a starting point for final 'street-level' exploration and search for optimal venue location by stakeholders.

#### First thing first ..lets get our libraries 

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geopy 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from geopy.extra.rate_limiter import RateLimiter 
from geopy import distance 

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
from matplotlib import pyplot as plt
%matplotlib inline

# import k-means from clustering stage
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report

!pip install yellowbrick
from yellowbrick.cluster import KElbowVisualizer

!pip install folium 
import folium # map rendering library
from folium.plugins import MarkerCluster

import requests 

from bs4 import BeautifulSoup 

! pip install googlemaps
import googlemaps

print('Libraries imported.')

Libraries imported.


### Preprocessing ...

please read this in a robot voice

#### Lets get our data 

Notice how the neighborhood also has the city and the country? Yup, Harare is in Zimbabwe, your guess was correct. 

Well, it is like this because I really didnt want to confuse the geocoder, my people have fancy naighborhoods that also exist in America and also South East China. 

In [2]:
harare = pd.read_csv(r'C:\Users\prosper.duwugwani\Desktop\projects\Coursera_Capstone\Harare Suburbs v2.csv')

df = pd.DataFrame(harare)

(180, 2)

In [3]:
df.head()

,Borough,Neighborhood
0,Budiriro,"Budiriro 1,Harare,Zimbabwe"
1,Budiriro,"Budiriro 3,Harare,Zimbabwe"
2,Budiriro,"Budiriro 2,Harare,Zimbabwe"
3,Budiriro,"Budiriro 5,Harare,Zimbabwe"
4,Budiriro,"Budiriro 4,Harare,Zimbabwe"


#### The infamous geocoder 

This is the part i used magic to get the latitude and longitude data. I say magic because, you give it a name of a place (for example, the tv remote under the couche) and it gives you a list of coodianates!! How awesome is that? So Awesome !!

In [5]:
gmaps_key = googlemaps.Client(key = "xxxxxxxxxxxxxxxxxx")

In [6]:
df["lat"] = None
df["lng"] = None

for i in range(len(df)):
    geocode_result = gmaps_key.geocode(df.loc[i,'Neighborhood'])
    try:
        lat = geocode_result[0]["geometry"]["location"]["lat"]
        lng = geocode_result[0]["geometry"]["location"]["lng"]
        df.loc[i,'lat'] = lat
        df.loc[i,'lng'] = lng
    except:
        lat = None
        lon = None
        
df

,Borough,Neighborhood,lat,lng
0,Budiriro,"Budiriro 1,Harare,Zimbabwe",-17.900426,30.921771
1,Budiriro,"Budiriro 3,Harare,Zimbabwe",-17.898069,30.924164
2,Budiriro,"Budiriro 2,Harare,Zimbabwe",-17.879094,30.935077
3,Budiriro,"Budiriro 5,Harare,Zimbabwe",-17.880828,30.932598
4,Budiriro,"Budiriro 4,Harare,Zimbabwe",-17.889796,30.935999
5,Dzivarasekwa,"Kuwdzana Park,Harare,Zimbabwe",-17.823438,31.047127
6,Dzivarasekwa,"Cold Comfort,Harare,Zimbabwe",-17.839628,30.947775
7,Dzivarasekwa,"Tynwald South,Harare,Zimbabwe",-17.806393,30.944606
8,Dzivarasekwa,"Glaudina ,Harare,Zimbabwe",-17.816536,30.897614
9,Dzivarasekwa,"Warren Park North,Harare,Zimbabwe",-17.832069,30.979028


In [2]:
df.dtypes

NameError: name 'df' is not defined

I didnt expect the data to be in the fomat that it is, so im going to do some nice fomating to get it ready for out model


In [8]:
df.rename(columns = {'lat': 'Latitude'}, inplace = True )
df.rename(columns = {'lng': 'Longitude'}, inplace = True )


In [10]:
df['Latitude'] = df.Latitude.astype(float)


In [11]:
df['Longitude'] = df.Longitude.astype(float)

In [26]:
df.dropna(how = 'any', inplace = True)

In [27]:
df.dtypes


Borough          object
Neighborhood     object
Latitude        float64
Longitude       float64
dtype: object

In [29]:
df.isnull().sum()

Borough         0
Neighborhood    0
Latitude        0
Longitude       0
dtype: int64

#### Map of Harare

I grew up in Harare, and i didnt even know where most of this neighborhoods are! They are just so many !!

So i used the folium library to visualize geographic details of Harare with the neighborhoods superimposed on top. I used latitude and longitude values to get the awesome map !!

In [34]:
address = 'Harare'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Harare are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Harare are -17.831773, 31.045686.


In [63]:
# create map of Harare using latitude and longitude values
map_harare = folium.Map(location=[-17.831773, 31.045686], zoom_start=10)

makerCluster = MarkerCluster().add_to(map_harare)

# add markers to map
for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(makerCluster )  
    
map_harare

#### [][][][] Foursquare

...the jokes literay writes themselves 

haha that was dry...
Okay, moving on. 
Let put the new location data we have into a new dataframe that we will use to get venue infomation from Foursquare

In [38]:
neighborhood_latitude = df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Budiriro 1,Harare,Zimbabwe are -17.9004264, 30.9217711.


#### Lets set our credentials 

please dont steal my passwords ...


In [6]:
CLIENT_ID = 'xxxxxxxx' # your Foursquare ID
CLIENT_SECRET = 'xxxxxx' # your Foursquare Secret
ACCESS_TOKEN = 'Axxxxx' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: xxxxxxxx
CLIENT_SECRET:xxxxxx


#### Now lets get the venues in out neighborhoods

In [40]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Lets create a new dataframe with the results from the code above

In [41]:
harare_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Budiriro 1,Harare,Zimbabwe
Budiriro 3,Harare,Zimbabwe
Budiriro 2,Harare,Zimbabwe
Budiriro 5,Harare,Zimbabwe
Budiriro 4,Harare,Zimbabwe
Kuwdzana Park,Harare,Zimbabwe
Cold Comfort,Harare,Zimbabwe
Tynwald South,Harare,Zimbabwe
Glaudina ,Harare,Zimbabwe
Warren Park North,Harare,Zimbabwe
Gevstein Park,Harare,Zimbabwe
Warren Par,Harare,Zimbabwe
Dzivarasekwa,Harare,Zimbabwe
Glen Norah,Harare,Zimbabwe
Ardbennie ,Harare,Zimbabwe
Glen View ,Harare,Zimbabwe
Kensington,Harare,Zimbabwe
Strathaven,Harare,Zimbabwe
Coronation Park,Harare,Zimbabwe
Eastlea North,Harare,Zimbabwe
Eastlea South,Harare,Zimbabwe
Ridgeview,Harare,Zimbabwe
New Ridgeview,Harare,Zimbabwe
Gun Hill,Harare,Zimbabwe
Civic Centre,Harare,Zimbabwe
Workington,Harare,Zimbabwe
Avondale,Harare,Zimbabwe
Belgravia,Harare,Zimbabwe
Avondale West,Harare,Zimbabwe
Avenues,Harare,Zimbabwe
Alexandra Park,Harare,Zimbabwe
Alexandra Park,Harare,Zimbabwe
Lincoln Green,Harare,Zimbabwe
Little Norfolk,Harare,Zimbabwe
Belvedere North,Harare,Zimbabwe
Monova

#### To see how what got returned we run the code

In [42]:
print(harare_venues.shape)
harare_venues.head()

(158, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Budiriro 2,Harare,Zimbabwe",-17.879094,30.935077,Mehmetçiğin yeri,-17.879015,30.933882,Airport
1,"Budiriro 5,Harare,Zimbabwe",-17.880828,30.932598,Mehmetçiğin yeri,-17.879015,30.933882,Airport
2,"Kuwdzana Park,Harare,Zimbabwe",-17.823438,31.047127,Harare Gardens,-17.825330,31.047600,Performing Arts Venue
3,"Kuwdzana Park,Harare,Zimbabwe",-17.823438,31.047127,Nssa House,-17.824842,31.049710,Shopping Mall
4,"Kuwdzana Park,Harare,Zimbabwe",-17.823438,31.047127,TechWorld PrintShop,-17.824514,31.049456,Print Shop


### Let's check how many venues were returned for each neighborhood

In [43]:
harare_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Alexandra Park,Harare,Zimbabwe",2,2,2,2,2,2
"Arcadia,Harare,Zimbabwe",1,1,1,1,1,1
"Ardbennie ,Harare,Zimbabwe",2,2,2,2,2,2
"Ashbrittle,Harare,Zimbabwe",1,1,1,1,1,1
"Athlone,Harare,Zimbabwe",2,2,2,2,2,2
"Avenues,Harare,Zimbabwe",2,2,2,2,2,2
"Avondale West,Harare,Zimbabwe",4,4,4,4,4,4
"Avondale,Harare,Zimbabwe",4,4,4,4,4,4
"Ballantyne Park,Harare,Zimbabwe",4,4,4,4,4,4


#### Let's find out how many unique categories can be curated from all the returned venues

In [44]:
print('There are {} uniques categories.'.format(len(harare_venues['Venue Category'].unique())))

There are 65 uniques categories.


#### Lets put things in a language everyone understands 

As you know, computers dont speak English (haha) so we are going to get dummy variables to represent the different venues we have.

So many 0s and 1s !

In [45]:
# one hot encoding
harare_onehot = pd.get_dummies(harare_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
harare_onehot['Neighborhood'] = harare_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [harare_onehot.columns[-1]] + list(harare_onehot.columns[:-1])
harare_onehot = harare_onehot[fixed_columns]

harare_onehot.head()

,Neighborhood,Airport,Arepa Restaurant,Art Museum,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bar,Bed & Breakfast,Beer Garden,Boat or Ferry,Breakfast Spot,Café,Chinese Restaurant,Coffee Shop,Convenience Store,Cosmetics Shop,Diner,Electronics Store,Farm,Fast Food Restaurant,Flea Market,Food Court,French Restaurant,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gastropub,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,History Museum,Home Service,Hotel,Indian Restaurant,Liquor Store,Mediterranean Restaurant,Movie Theater,Multiplex,Museum,Music Venue,Nature Preserve,Park,Performing Arts Venue,Pizza Place,Plaza,Pool Hall,Portuguese Restaurant,Print Shop,Rental Car Location,Restaurant,Shoe Store,Shop & Service,Shopping Mall,Soccer Field,Sports Bar,Steakhouse,Supermarket,Tapas Restaurant,Tennis Court,Thai Restaurant,Tour Provider,Turkish Restaurant
0,"Budiriro 2,Harare,Zimbabwe",1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Budiriro 5,Harare,Zimbabwe",1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Kuwdzana Park,Harare,Zimbabwe",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Kuwdzana Park,Harare,Zimbabwe",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,"Kuwdzana Park,Harare,Zimbabwe",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [46]:
harare_onehot.shape

(158, 66)

#### Lets group the neighborhoods with multiple places to hang out

In [47]:
harare_grouped = harare_onehot.groupby('Neighborhood').mean().reset_index()
harare_grouped

,Neighborhood,Airport,Arepa Restaurant,Art Museum,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bar,Bed & Breakfast,Beer Garden,Boat or Ferry,Breakfast Spot,Café,Chinese Restaurant,Coffee Shop,Convenience Store,Cosmetics Shop,Diner,Electronics Store,Farm,Fast Food Restaurant,Flea Market,Food Court,French Restaurant,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gastropub,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,History Museum,Home Service,Hotel,Indian Restaurant,Liquor Store,Mediterranean Restaurant,Movie Theater,Multiplex,Museum,Music Venue,Nature Preserve,Park,Performing Arts Venue,Pizza Place,Plaza,Pool Hall,Portuguese Restaurant,Print Shop,Rental Car Location,Restaurant,Shoe Store,Shop & Service,Shopping Mall,Soccer Field,Sports Bar,Steakhouse,Supermarket,Tapas Restaurant,Tennis Court,Thai Restaurant,Tour Provider,Turkish Restaurant
0,"Alexandra Park,Harare,Zimbabwe",0.0,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1.00,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000,0.000000,0.000000,0.0,0.00
1,"Arcadia,Harare,Zimbabwe",0.0,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,0.00,0.0,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000,0.000000,0.000000,0.0,0.00
2,"Ardbennie ,Harare,Zimbabwe",0.0,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,1.000000,0.00,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000,0.000000,0.000000,0.0,0.00
3,"Ashbrittle,Harare,Zimbabwe",0.0,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,1.0,0.00,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000,0.000000,0.000000,0.0,0.00
4,"Athlone,Harare,Zimbabwe",0.0,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.5,0.0,0.000000,0.500000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000,0.000000,0.000000,0.0,0.00
5,"Avenues,Harare,Zimbabwe",0.0,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.5,0.000000,0.000000,0.500000,0.00,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000,0.000000,0.000000,0.0,0.00
6,"Avondale West,Harare,Zimbabwe",0.0,0.25,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.5,0.000000,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.000000,0.0,0.25,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000,0.000000,0.000000,0.0,0.00
7,"Avondale,Harare,Zimbabwe",0.0,0.00,0.000000,0.0,0.

#### Let's confirm the new size

In [48]:
harare_grouped.shape

(67, 66)

#### Let's print each neighborhood along with the top 5 most common venues

We are on our way to get the first objective out of the way! This is so exciting 

In [4]:
num_top_venues = 5

for hood in harare_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = harare_grouped[harare_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

NameError: name 'harare_grouped' is not defined

#### Let's put that into a pandas dataframe

Notices we are changing things around so that you can also understands and get in on the action? Yup, you already noticed that, you are so smart !!

In [50]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [51]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = harare_grouped['Neighborhood']

for ind in np.arange(harare_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(harare_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Alexandra Park,Harare,Zimbabwe",Garden,Airport,Pool Hall,Hotel,Indian Restaurant,Liquor Store,Mediterranean Restaurant,Movie Theater,Multiplex,Museum
1,"Arcadia,Harare,Zimbabwe",Fast Food Restaurant,Airport,Pool Hall,Hotel,Indian Restaurant,Liquor Store,Mediterranean Restaurant,Movie Theater,Multiplex,Museum
2,"Ardbennie ,Harare,Zimbabwe",Portuguese Restaurant,History Museum,Hotel,Indian Restaurant,Liquor Store,Mediterranean Restaurant,Movie Theater,Multiplex,Museum,Music Venue
3,"Ashbrittle,Harare,Zimbabwe",Furniture / Home Store,Airport,Pool Hall,Hotel,Indian Restaurant,Liquor Store,Mediterranean Restaurant,Movie Theater,Multiplex,Museum
4,"Athlone,Harare,Zimbabwe",Music Venue,Performing Arts Venue,Airport,History Museum,Hotel,Indian Restaurant,Liquor Store,Mediterranean Restaurant,Movie Theater,Multiplex
5,"Avenues,Harare,Zimbabwe",Portuguese Restaurant,Pizza Place,History Museum,Hotel,Indian Restaurant,Liquor Store,Mediterranean Restaurant,Movie Theater,Multiplex,Museum
6,"Avondale West,Harare,Zimbabwe",Golf Course,Arepa Restaurant,Movie Theater,Pool Hall,Hotel,Indian Restaurant,Liquor Store,Mediterranean Restaurant,Multiplex,Museum
7,"Avondale,Harare,Zimbabwe",Health & Beauty Service,Cosmetics Shop,Garden,Turkish Restaurant,Athletics & Sports,Hotel,Liquor Store,Mediterranean Restaurant,Movie Theater,Multiplex
8,"Ballantyne Park,Harare,Zimbabwe",Steakhouse,Grocery Store,Bakery,Bed & Breakfast,Museum,Pizza Place,Performing Arts Venue,Park,Nature Preserve,Music Venue
9,"Belgravia,Harare,Zimbabwe",Café,Diner,Grocery Store,Steakhouse,Music Venue,Plaza,Pizza Place,Performing Arts Venue,Park,Nature Preserve


### Data Analysis

We are qoing to use the elbow method to find the optimal k for our model 

In [ ]:
k_range = range(1,10)
sse = []

for k in k_range:
    km = KMeans(n_clusters = k, random_state=0)
    km.fit(toronto_grouped.drop('Neighborhood', 1))
    sse.append(km.inertia_)

In [ ]:
plt.xlabel('K')
plt.ylabel('Sum of Squared Error')
plt.plot(k_range,sse)

#### Cluster Neighborhoods

We get 4 as out optimal K

In [84]:

# set number of clusters
kclusters = 4

harare_grouped_clustering = harare_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(harare_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 3, 1, 3, 4, 1, 0, 3, 3, 3])

#### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [82]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

harare_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
harare_merged = harare_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

ValueError: cannot insert Cluster Labels, already exists

In [ ]:
harare_merged.dtypes

In [85]:
harare_merged['Cluster Labels'] = pd.to_numeric(harare_merged['Cluster Labels'].round(0), errors='coerce', downcast = 'integer')

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Budiriro,"Budiriro 1,Harare,Zimbabwe",-17.900426,30.921771,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Budiriro,"Budiriro 3,Harare,Zimbabwe",-17.898069,30.924164,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Budiriro,"Budiriro 2,Harare,Zimbabwe",-17.879094,30.935077,3.0,Airport,History Museum,Hotel,Indian Restaurant,Liquor Store,Mediterranean Restaurant,Movie Theater,Multiplex,Museum,Music Venue
3,Budiriro,"Budiriro 5,Harare,Zimbabwe",-17.880828,30.932598,3.0,Airport,History Museum,Hotel,Indian Restaurant,Liquor Store,Mediterranean Restaurant,Movie Theater,Multiplex,Museum,Music Venue
4,Budiriro,"Budiriro 4,Harare,Zimbabwe",-17.889796,30.935999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Finally, let's visualize the resulting clusters

In [81]:
# create map
map_clusters = folium.Map(location=[-17.831773, 31.045686], tiles = 'OpenStreetMap', zoom_start=11)

makerCluster = MarkerCluster().add_to(map_clusters)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(harare_merged['Latitude'], harare_merged['Longitude'], harare_merged['Neighborhood'], harare_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        #color=rainbow[cluster-1],
        fill=True,
        #fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(makerCluster)
       
map_clusters

### Results and Discussion 

The battel of a neighborhood project explored different categories of venues. In this analysis, you will notice that most affluent surbubs have many amenities and access to most places to go. 

I examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, one can then assign a name to each cluster from the table which also has the below information for each neighborhood.


#### Examine Cluster 1

In [57]:
harare_merged.loc[harare_merged['Cluster Labels'] == 0, harare_merged.columns[[1] + list(range(5, harare_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,"Eastlea South,Harare,Zimbabwe",Golf Course,Airport,Pool Hall,Hotel,Indian Restaurant,Liquor Store,Mediterranean Restaurant,Movie Theater,Multiplex,Museum
28,"Avondale West,Harare,Zimbabwe",Golf Course,Arepa Restaurant,Movie Theater,Pool Hall,Hotel,Indian Restaurant,Liquor Store,Mediterranean Restaurant,Multiplex,Museum
60,"Brooke Ridge,Harare,Zimbabwe",Golf Course,Shoe Store,Airport,Museum,Pizza Place,Performing Arts Venue,Park,Nature Preserve,Music Venue,Multiplex
61,"Borrowdale Brooke,Harare,Zimbabwe",Athletics & Sports,Golf Course,Airport,Music Venue,Plaza,Pizza Place,Performing Arts Venue,Park,Nature Preserve,Multiplex


#### Examine Cluster 2

In [59]:
harare_merged.loc[harare_merged['Cluster Labels'] == 1, harare_merged.columns[[1] + list(range(5, harare_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,"Ardbennie ,Harare,Zimbabwe",Portuguese Restaurant,History Museum,Hotel,Indian Restaurant,Liquor Store,Mediterranean Restaurant,Movie Theater,Multiplex,Museum,Music Venue
29,"Avenues,Harare,Zimbabwe",Portuguese Restaurant,Pizza Place,History Museum,Hotel,Indian Restaurant,Liquor Store,Mediterranean Restaurant,Movie Theater,Multiplex,Museum
53,"Highlands,Harare,Zimbabwe",Pizza Place,Airport,History Museum,Hotel,Indian Restaurant,Liquor Store,Mediterranean Restaurant,Movie Theater,Multiplex,Museum
75,"Runniville,Harare,Zimbabwe",Pizza Place,Airport,History Museum,Hotel,Indian Restaurant,Liquor Store,Mediterranean Restaurant,Movie Theater,Multiplex,Museum
76,"Colne Valley,Harare,Zimbabwe",Pizza Place,Airport,History Museum,Hotel,Indian Restaurant,Liquor Store,Mediterranean Restaurant,Movie Theater,Multiplex,Museum
169,"Ardbennie ,Harare,Zimbabwe",Portuguese Restaurant,History Museum,Hotel,Indian Restaurant,Liquor Store,Mediterranean Restaurant,Movie Theater,Multiplex,Museum,Music Venue


#### Examine Cluster 3

In [60]:
harare_merged.loc[harare_merged['Cluster Labels'] == 2, harare_merged.columns[[1] + list(range(5, harare_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
82,"Ryelands,Harare,Zimbabwe",Convenience Store,Café,Airport,Nature Preserve,Plaza,Pizza Place,Performing Arts Venue,Park,Music Venue,Portuguese Restaurant
114,"Greencroft,Harare,Zimbabwe",Convenience Store,Airport,Music Venue,Plaza,Pizza Place,Performing Arts Venue,Park,Nature Preserve,Museum,Portuguese Restaurant
115,"Mabelreign,Harare,Zimbabwe",Sports Bar,Convenience Store,Museum,Pizza Place,Performing Arts Venue,Park,Nature Preserve,Music Venue,Airport,Pool Hall
132,"Cranbourne Park,Harare,Zimbabwe",Convenience Store,Airport,Music Venue,Plaza,Pizza Place,Performing Arts Venue,Park,Nature Preserve,Museum,Portuguese Restaurant
153,"Rugare ,Harare,Zimbabwe",Convenience Store,Airport,Music Venue,Plaza,Pizza Place,Performing Arts Venue,Park,Nature Preserve,Museum,Portuguese Restaurant


#### Examine Cluster 4

In [61]:
harare_merged.loc[harare_merged['Cluster Labels'] == 3, harare_merged.columns[[1] + list(range(5, harare_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Budiriro 2,Harare,Zimbabwe",Airport,History Museum,Hotel,Indian Restaurant,Liquor Store,Mediterranean Restaurant,Movie Theater,Multiplex,Museum,Music Venue
3,"Budiriro 5,Harare,Zimbabwe",Airport,History Museum,Hotel,Indian Restaurant,Liquor Store,Mediterranean Restaurant,Movie Theater,Multiplex,Museum,Music Venue
5,"Kuwdzana Park,Harare,Zimbabwe",Print Shop,Performing Arts Venue,Breakfast Spot,Shopping Mall,Music Venue,Pizza Place,Park,Nature Preserve,Museum,Pool Hall
17,"Strathaven,Harare,Zimbabwe",Bar,Airport,Museum,Plaza,Pizza Place,Performing Arts Venue,Park,Nature Preserve,Music Venue,Multiplex
18,"Coronation Park,Harare,Zimbabwe",Hotel,Airport,History Museum,Indian Restaurant,Liquor Store,Mediterranean Restaurant,Movie Theater,Multiplex,Museum,Music Venue
19,"Eastlea North,Harare,Zimbabwe",Indian Restaurant,Airport,History Museum,Hotel,Liquor Store,Mediterranean Restaurant,Movie Theater,Multiplex,Museum,Music Venue
24,"Civic Centre,Harare,Zimbabwe",Restaurant,Hotel,Gym / Fitness Center,Asian Restaurant,Music Venue,Pizza Place,Performing Arts Venue,Park,Nature Preserve,Multiplex
25,"Workington,Harare,Zimbabwe",History Museum,Airport,Hotel,Indian Restaurant,Liquor Store,Mediterranean Restaurant,Movie Theater,Multiplex,Museum,Music Venue
26,"Avondale,Harare,Zimbabwe",Health & Beauty Service,Cosmetics Shop,Garden,Turkish Restaurant,Athletics & Sports,Hotel,Liquor Store,Mediterranean Restaurant,Movie Theater,Multiplex
27,"Belgravia,Harare,Zimbabwe",Café,Diner,Grocery Store,Steakhouse,Music Venue,Plaza,Pizza Place,Performing Arts Venue,Park,Nature Preserve


### Conclusion 

Final decisions on optimal restaurant location will be made by stakeholders based on specific characteristics of neighborhoods and locations in every recommended zone, taking into consideration additional factors like attractiveness of each location (proximity to park or water), levels of noise/proximity to major roads, real estate availability, prices, social and economic dynamics of every neighborhood.

As a result, people are turning to big cities to start a business or work. For this reason, people can achieve better outcomes through their access to the platforms where such information is provided.

Not only for investors but also city managers can manage the city more regularly by using similar data analysis types or platforms